In [19]:
import pyspark
import numpy as np
import pandas as pd
import os
from os import walk
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import *
import time
from pyspark.sql.functions import date_format
from datetime import date
import datetime
from datetime import datetime
import re

date = date(2023,2,27)                                     ## TO BE CHANGED DAILY AS PER UPDATION DATE
day = date.strftime('%d')
nummonth=date.strftime("%m")
year=date.strftime('%Y')
tablename="r"+str(day)+str(nummonth)+str(year)
print(tablename)

st=time.time()

def banknifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.0.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()

    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    bndata = q.filter(q.ticker.contains('BANKNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    bndata = bndata.withColumn('ticker',regexp_replace('ticker','.NFO',''))

    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    bndata = bndata.toPandas()
    return bndata

def banknifty_monthly():
    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Monthly_data\\"
    sym = 'BANKNIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()
    
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    ## READING THE EXPIRY SHEET
    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates=["curr_exp_date","curr_date"],dayfirst=True).dropna()
    ## CONVERTING TO PANDAS DATAFRAME
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    exp_df['New_date'] = pd.to_datetime(exp_df['New_date'],dayfirst=True)
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')
    
    ## CALLING FUNCTION BANKNIFTY_DATA TO GENERATE ONLY BANKNIFTY TICKERS
    ddf = banknifty_data()
    ddf = ddf.loc[:, ~ddf.columns.str.contains('^Unnamed')]
    ddf['date'] = pd.to_datetime(ddf['date'],dayfirst=True)
    ddf['Optiontype'] = ddf['ticker'].str[-2:]
    ddf['Temp'] = ddf['ticker'].str.replace('BANKNIFTY','')
    ddf['Temp'] = ddf['Temp'].str[:-2]
    ddf['Length_of_temp'] = np.where(ddf['Temp'].str.len()==12,12,ddf['Temp'].str.len())
    ddf['Strike'] = np.where((ddf['Temp'].str.len()==12)|(ddf['Temp'].str.len()==10),ddf['Temp'].str[-5:],
                             ddf['Temp'].str[-4:])
    ddf['Exp_year'] = np.where(ddf['Temp'].str.len()==12,ddf['Temp'].str[5:7],ddf['Temp'].str[:2])
    ddf['Exp_month'] = ddf['Temp'].str[2:5]
    ddf['Exp_year'] = ddf['Exp_year'].astype('str')
    ddf['MonthYear'] = ddf['Exp_month'] + ddf['Exp_year']
    merged_df = pd.merge(ddf,exp_date,on='MonthYear')
    merged_df = merged_df.drop(['MonthYear','Month','Year'],axis=1)
    merged_df['Length_of_temp'] = merged_df['Length_of_temp'].astype('int64')
    df_10 = merged_df[(merged_df['Length_of_temp']==10) | (merged_df['Length_of_temp']==9)]
    df_12 = merged_df[merged_df['Length_of_temp']==12]
    df_12['DateDate'] = df_12['Temp'].str[:2]
    df_12['DateDate'] = df_12['DateDate'].astype('int64')
    df_12['Exp_DT'] = pd.to_datetime(merged_df['Exp_DT'],dayfirst=True)
    df_12['Exp_Day'] = df_12['Exp_DT'].dt.day
    df_12 = df_12[df_12['Exp_Day']==df_12['DateDate']]
    df_12 = df_12.drop(['DateDate','Exp_Day'],axis=1)

    ddf = df_10.append(df_12,ignore_index=True)
    ddf['time'] = ddf['time'].str.replace(' 15:00:59','15:00:59')
    ddf['time'] = ddf['time'].str.replace(' 9:','09:',regex=True)
    ddf['time'] = pd.to_datetime(ddf['time'], format='%H:%M:%S').dt.time
    ddf = ddf[(ddf['time']>=start_time) & (ddf['time']<=end_time)]
    ddf['exp_month_number'] = ddf.apply(lambda row : add(row["Exp_month"]), axis = 1)
    ddf['New_date'] = ddf['date']
    ddf["New_date"] = pd.to_datetime(ddf["New_date"],dayfirst=True)
    ddf["current_month_number"] = ddf['New_date'].dt.month
    ddf["difference"] = ddf['exp_month_number'].astype(int) - ddf["current_month_number"].astype(int)
    df1 = pd.merge(ddf, 
                     exp_df, 
                     on ='New_date', 
                     how ='left')
    df1.drop(df1.filter(regex="Unname"),axis=1, inplace=True)
    df1["current_exp_month_number"] = df1['curr_exp_date'].dt.month
    df1["Diff_months"] = df1["current_exp_month_number"] - df1["current_month_number"]
    df1["Diff_months"] = df1["Diff_months"].astype(int) 
    bdf = df1[df1["Diff_months"] == 0]
    adf = df1[(df1["Diff_months"] == 1) | (df1["Diff_months"] == -11)]
    if bdf.shape[0] + adf.shape[0] == df1.shape[0]:
        print("Sanity Check Success")
    else:
        print("Error1")
    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())
    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())
    
    ## REMOVING YESTERDAY'S CREATED FILE
    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'misc.csv'):
        os.remove(folpath+sym+'misc.csv')
        
    for i in unique_val_list_b:
        temp_df_new = bgb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 0:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)
        
    for i in unique_val_list_a:
        temp_df_new = agb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i == 3 or i == -9:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)
    
    ##########################CREATING LABEL IN STANDARD FORM#####################
    for i in range(3):
        if i == 0:
            file='I'
        elif i == 1:
            file='II'
        elif i == 2:
            file='III'
        
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\BankNifty\Monthly\Banknifty-'+file+".csv"):
            os.remove(r'C:\users\admin\desktop\Pyspark\BankNifty\Monthly\Banknifty-'+file+".csv")
    
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Monthly_Data\BANKNIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Monthly_Data\BANKNIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'MONTHLY-' + file + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {"ticker":"Ticker"})
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Monthly\\Banknifty-"+file+".csv",index=False)
    print("BANKNIFTY MONTHLY CONTRACTS CREATED")

def banknifty_weekly():
    folpath = r"C:\users\admin\desktop\\Pyspark_Contracts\\BankNifty\\Weekly_Data\\"
    sym = 'BANKNIFTY'
    s = 'BANKNIFTY'
    expiry_time = datetime.strptime('15:29:59', '%H:%M:%S').time()
    
    ## CALLING FUNCTION NIFTY_DATA TO GENERATE ONLY NIFTY TICKERS
    df=banknifty_data()
    
    ## READING WEEKLY EXPIRY FILES
    exp_df = pd.read_csv(r"C:\Users\admin\Downloads\WeeklyExpiry.csv",parse_dates = ["date"],dayfirst =True,usecols= ['date', 'Week_number'])
    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx',parse_dates = ['Exp_DT'],usecols = ['MonthYear','Exp_DT'])
    df['date'] = pd.to_datetime(df['date'])
    df['time'] = df['time'].str.replace(' 15:00:59','15:00:59')
    df['time'] = df['time'].str.replace(' 9:','09:',regex=True)
    df['time'] = pd.to_datetime(df['time']).dt.time
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df[df['time'] <= expiry_time]
    df['EXPIRY_DT'] = df['ticker'].str[9:16]
    df['EXPIRY_DT'] = pd.to_datetime(df['EXPIRY_DT'],dayfirst=True)
    df['OPTION_TYP'] = df['ticker'].str[-2:]
    df['STRIKE_PR'] = np.where(df['ticker'].str.len()==23,df['ticker'].str[-7:-2],df['ticker'].str[-7:-2])
    df['Month'] = df['ticker'].str[11:14]
    df['Year'] = np.where(df['ticker'].str.len()==23,df['ticker'].str[14:16],df['ticker'].str[9:11])
    df['MonthYear'] = df['Month'] + df['Year']
    df = df.rename(columns={'EXPIRY_DT' : 'expiry_date'})

    ## MERGING WITH EXPIRY SHEET TO GET EXPIRY DATE
    df1 = pd.merge(df,exp_df,on='date',how='left')
    df1 = df1.drop_duplicates()
    df1 = pd.merge(df1,exp_date,on='MonthYear',how='left')
    df1 = df1.drop(['Month','Year','MonthYear'],axis=1)

    ## GETTING THE EXPIRY DATES FOR MONTHLY CONTRACTS
    df1['expiry_date'] = np.where(df1['ticker'].str.len()>21,df1['expiry_date'],df1['Exp_DT'])
    df1 = df1.drop(['Exp_DT'],axis=1)
    exp_df = pd.read_csv(r"C:\Users\admin\Downloads\WeeklyExpiry.csv",parse_dates = ["Weekly_Expiry_Date"],dayfirst =True,usecols= ['Weekly_Expiry_Date', 'Expiry_Week_number'])
    exp_df = exp_df.dropna()
    exp_df = exp_df.rename(columns = {'Weekly_Expiry_Date': 'expiry_date'})
    df2 = pd.merge(df1, exp_df, on ='expiry_date', how ='left')
    df2 = df2.drop_duplicates()
    df2['week_diff'] = df2['Expiry_Week_number'] - df2['Week_number']

    final_df = df2[(df2["OPTION_TYP"] == "CE") | (df2["OPTION_TYP"] == "PE") ]
    final_df["week_diff"] = final_df['week_diff'].replace(np.nan,10000)

    agb = final_df.groupby(["week_diff"])
    unique_val_list_a = list(final_df["week_diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:12]
    print(unique_val_list_a)

    ## CREATING -I,-II AND SO ON BASED ON THE WEEK DIFFERENCES

    if os.path.exists(folpath+sym+'_Weekly-I.csv'):
        os.remove(folpath+sym+'_Weekly-I.csv')
    if os.path.exists(folpath+sym+'_Weekly-II.csv'):
        os.remove(folpath+sym+'_Weekly-II.csv')
    if os.path.exists(folpath+sym+'_Weekly-III.csv'):
        os.remove(folpath+sym+'_Weekly-III.csv')
    if os.path.exists(folpath+sym+'_Weekly-IV.csv'):
        os.remove(folpath+sym+'_Weekly-IV.csv')
    if os.path.exists(folpath+sym+'_Weekly-V.csv'):
        os.remove(folpath+sym+'_Weekly-V.csv')
    if os.path.exists(folpath+sym+'_Weekly-VI.csv'):
        os.remove(folpath+sym+'_Weekly-VI.csv')
    if os.path.exists(folpath+sym+'_Weekly-VII.csv'):
        os.remove(folpath+sym+'_Weekly-VII.csv')
    if os.path.exists(folpath+sym+'_Weekly-VIII.csv'):
        os.remove(folpath+sym+'_Weekly-VIII.csv')
    if os.path.exists(folpath+sym+'_Weekly-IX.csv'):
        os.remove(folpath+sym+'_Weekly-IX.csv')
    if os.path.exists(folpath+sym+'_Weekly-X.csv'):
        os.remove(folpath+sym+'_Weekly-X.csv')
    if os.path.exists(folpath+sym+'_Weekly-XI.csv'):
        os.remove(folpath+sym+'_Weekly-XI.csv')
    if os.path.exists(folpath+sym+'_Weekly-XII.csv'):
        os.remove(folpath+sym+'_Weekly-XII.csv')  
    if os.path.exists(folpath+sym+'_Weekly-XIII.csv'):
        os.remove(folpath+sym+'_Weekly-XIII.csv')  
    if os.path.exists(folpath+sym+'_Weekly-XIV.csv'):
        os.remove(folpath+sym+'_Weekly-XIV.csv')  
        
    for i in sorted(unique_val_list_a):
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        temp_df = temp_df.drop_duplicates()
        if i == 0:
            temp_df.to_csv(folpath + s + '-I.csv', mode = 'a', header = not os.path.exists(folpath + s + '-I.csv'), index=False)
        if i == 1:
            temp_df.to_csv(folpath + s + '-II.csv', mode = 'a', header = not os.path.exists(folpath + s + '-II.csv'), index=False)
        if i == 2:
            temp_df.to_csv(folpath + s + '-III.csv', mode = 'a', header = not os.path.exists(folpath + s + '-III.csv'), index=False)
        if i == 3:
            temp_df.to_csv(folpath + s + '-IV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IV.csv'), index=False)
        if i == 4:
            temp_df.to_csv(folpath + s + '-V.csv', mode = 'a', header = not os.path.exists(folpath + s + '-V.csv'), index=False)
        if i == 5:
            temp_df.to_csv(folpath + s + '-VI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VI.csv'), index=False)
        if i == 6:
            temp_df.to_csv(folpath + s + '-VII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VII.csv'), index=False)
        if i == 7:
            temp_df.to_csv(folpath + s + '-VIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VIII.csv'), index=False)
        if i == 8:
            temp_df.to_csv(folpath + s + '-IX.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IX.csv'), index=False)
        if i == 9:
            temp_df.to_csv(folpath + s + '-X.csv', mode = 'a', header = not os.path.exists(folpath + s + '-X.csv'), index=False)
        if i == 10:
            temp_df.to_csv(folpath + s + '-XI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XI.csv'), index=False)
        if i == 11:
            temp_df.to_csv(folpath + s + '-XII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XII.csv'), index=False)
        if i == 12:
            temp_df.to_csv(folpath + s + '-XIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIII.csv'), index=False)
        if i == 13:
            temp_df.to_csv(folpath + s + '-XIV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIV.csv'), index=False)
    
    #################LABELLING FILES IN STANDARD FORM######################
    for i in range(15):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        elif i==4:
            file='V'
        elif i==5:
            file='VI'
        elif i==6:
            file='VII'
        elif i==7:
            file='VIII'
        elif i==8:
            file='IX'
        elif i==9:
            file='X'
        elif i==10:
            file='XI'
        elif i==11:
            file='XII'
        elif i==12:
            file='XIII'
        elif i==13:
            file='XIV'
        if os.path.exists(r'C:\users\admin\desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-'+file+'.csv'):
            os.remove(r'C:\users\admin\desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-'+file+'.csv')
        if os.path.exists(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Weekly_Data\BANKNIFTY-'+file+'.csv'):
            ddf = pd.read_csv(r'C:\users\admin\desktop\Pyspark_Contracts\BankNifty\Weekly_Data\BANKNIFTY-'+file+'.csv')
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'WEEKLY-' + file + + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Weekly\BANKNIFTY-"+file+".csv",index=False)
    print("BANKNIFTY WEEKLY CONTRACTS CREATED")
    
def banknifty_quarterly():
    folpath = r"C:\users\admin\desktop\\Pyspark_Contracts\\BankNifty\\Quarterly_Data\\"
    sym = 'BANKNIFTY'
    start_time = datetime.strptime('09:15:00', '%H:%M:%S').time()
    end_time = datetime.strptime('15:30:00', '%H:%M:%S').time()

    ## CALLING FUNCTION BANKNIFTY_DATA TO GENERATE ONLY BANKNIFTY TICKERS
    temp = banknifty_data()
    
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number

    exp_date = pd.read_excel(r'C:\users\admin\desktop\Expiry_DT.xlsx')    ## reading the expiry sheet file
    exp_file_path = r"C:\Users\admin\Downloads\MonthlyExpiry.csv"
    exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","curr_date"],dayfirst =True,usecols = ["curr_exp_date","curr_date"]).dropna()
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    temp['time'] = temp['time'].astype(str).str.replace(' 15:00:59','15:00:59')
    temp['time'] = temp['time'].str.replace(' 9:','09:',regex=True)
    temp['ticker'] = temp['ticker'].str.replace('30MAR23','29MAR23',regex=True)
    temp['time'] = pd.to_datetime(temp['time']).dt.time
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp[(temp['time']>=start_time) & (temp['time']<=end_time)]
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['Option_type'] = temp['ticker'].str[-2:]
    temp["Temp"] = temp["ticker"].str.replace('BANKNIFTY',"")
    temp["Temp"] = temp["Temp"].str[:-2]
    temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                temp['Temp'].str[-5:],
                                temp['Temp'].str[-4:])
    temp['Current_Year'] = temp['date'].dt.year
    temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
    temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
    temp["Exp_month"] = temp["Temp"].str[2:5]
    temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
    temp['Exp_year'] = temp['Exp_year'].astype('str')
    temp['MonthYear'] = temp['Exp_month']+temp['Exp_year']
    temp = pd.merge(temp,exp_date,on='MonthYear')
    temp = temp.drop(['MonthYear','Month','Year'],axis=1)

    temp['Length_of_Temp'] = temp['Length_of_Temp'].astype('int64')
    temp_10 = temp[(temp['Length_of_Temp']==10) | (temp['Length_of_Temp']==9)]
    temp_12 = temp[temp['Length_of_Temp']==12]
    temp_12['datedate'] = temp_12['Temp'].str[:2]
    temp_12['datedate'] = temp_12['datedate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp['Exp_DT'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['datedate']]

    temp_12 = temp_12.drop(['datedate','Exp_Day'],axis=1)
    temp = temp_10.append(temp_12,ignore_index=True)

    temp['exp_month_number'] = temp.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp['New_date'] = temp['date']
    temp["New_date"] = pd.to_datetime(temp["New_date"])
    temp["current_month_number"] = temp['New_date'].dt.month
    temp["difference"] = temp['exp_month_number'].astype(int) - temp["current_month_number"].astype(int)
    temp['Year_difference'] = temp['Exp_year'].astype(int) - temp['Current_Year'].astype(int)
    temp = temp[(temp['exp_month_number']==3) | (temp['exp_month_number']==6) | (temp['exp_month_number']==9) | (temp['exp_month_number']==12)]

    temp1 = pd.merge(temp, 
                         exp_df, 
                         on ='New_date', 
                         how ='left')

    temp1.drop(temp1.filter(regex="Unname"),axis=1, inplace=True)
    temp1["current_exp_month_number"] = temp1['curr_exp_date'].dt.month
    temp1["Diff_months"] = temp1["current_exp_month_number"] - temp1["current_month_number"]
    temp1["Diff_months"] = temp1["Diff_months"].astype(int) 

    temp1 = temp1[temp1['Exp_DT']>=temp1['curr_exp_date']]                 ## to filter out dates which have wrong ticker

    ## creating groups for generating contracts
    atemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==0)]
    agb = atemp.groupby(['difference'])
    unique_a = list(atemp['difference'].unique())

    if os.path.exists(folpath+sym+'-I.csv'):
        os.remove(folpath+sym+'-I.csv')
    if os.path.exists(folpath+sym+'-II.csv'):
        os.remove(folpath+sym+'-II.csv')
    if os.path.exists(folpath+sym+'-III.csv'):
        os.remove(folpath+sym+'-III.csv')
    if os.path.exists(folpath+sym+'-IV.csv'):
        os.remove(folpath+sym+'-IV.csv')
    if os.path.exists(folpath+sym+'-V.csv'):
        os.remove(folpath+sym+'-V.csv')
    if os.path.exists(folpath+sym+'-VI.csv'):
        os.remove(folpath+sym+'-VI.csv')
    if os.path.exists(folpath+sym+'-VII.csv'):
        os.remove(folpath+sym+'-VII.csv')
    if os.path.exists(folpath+sym+'-VIII.csv'):
        os.remove(folpath+sym+'-VIII.csv')    

    for i in unique_a:
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==0 or i==1 or i==2:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==3 or i==4 or i==5:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==6 or i==7 or i==8:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==9 or i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    btemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==1)]
    bgb = btemp.groupby(['difference'])
    unique_b = list(btemp['difference'].unique())        

    for i in unique_b:
        temp_df = bgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-7 or i==-8 or i==-9:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==-4 or i==-5 or i==-6:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==-1 or i==-2 or i==-3:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)
        if i==0 or i==1 or i==2:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==3 or i==4 or i==5:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==6 or i==7 or i==8:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==9 or i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ctemp = temp1[(temp1['Diff_months']==0) & (temp1['Year_difference']==2)]
    cgb = ctemp.groupby(['difference'])
    unique_c = list(ctemp['difference'].unique())        

    for i in unique_c:
        temp_df = cgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-7 or i==-8 or i==-9:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==-4 or i==-5 or i==-6:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==-1 or i==-2 or i==-3:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    dtemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==0)]
    dgb = dtemp.groupby(['difference'])
    unique_d = list(dtemp['difference'].unique())

    for i in unique_d:
        temp_df = dgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==1 or i==2 or i==3:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==4 or i==5 or i==6:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==7 or i==8 or i==9:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)

    etemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==1)]
    egb = etemp.groupby(['difference'])
    unique_e = list(etemp['difference'].unique())

    for i in unique_e:
        temp_df = egb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-9:
            temp_df.to_csv(folpath + sym + '-I.csv', mode='a', header=not os.path.exists(folpath + sym + '-I.csv'), index=False)
        if i==-6 or i==-7:
            temp_df.to_csv(folpath + sym + '-II.csv', mode='a', header=not os.path.exists(folpath + sym + '-II.csv'), index=False)
        if i==-3 or i==-4:
            temp_df.to_csv(folpath + sym + '-III.csv', mode='a', header=not os.path.exists(folpath + sym + '-III.csv'), index=False)
        if i==0 or i==-1 or i==-2:
            temp_df.to_csv(folpath + sym + '-IV.csv', mode='a', header=not os.path.exists(folpath + sym + '-IV.csv'), index=False)
        if i==1 or i==2 or i==3:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==4 or i==5 or i==6:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==7 or i==8 or i==9:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==10 or i==11:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)

    ftemp = temp1[((temp1['Diff_months']==1) | (temp1['Diff_months']==-11)) & (temp1['Year_difference']==2)]
    fgb = ftemp.groupby(['difference'])
    unique_f = list(ftemp['difference'].unique())

    for i in unique_f:
        temp_df = fgb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i==-9:
            temp_df.to_csv(folpath + sym + '-V.csv', mode='a', header=not os.path.exists(folpath + sym + '-V.csv'), index=False)
        if i==-6 or i==-7:
            temp_df.to_csv(folpath + sym + '-VI.csv', mode='a', header=not os.path.exists(folpath + sym + '-VI.csv'), index=False)
        if i==-3 or i==-4:
            temp_df.to_csv(folpath + sym + '-VII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VII.csv'), index=False)
        if i==0 or i==-1 or i==-2:
            temp_df.to_csv(folpath + sym + '-VIII.csv', mode='a', header=not os.path.exists(folpath + sym + '-VIII.csv'), index=False)
    
    ## CREATING THE TICKER AND REMOVING ADDITIONAL COLUMNS
    for i in range(4):
        if i==0:
            file='I'
        elif i==1:
            file='II'
        elif i==2:
            file='III'
        elif i==3:
            file='IV'
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv"):
            os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv")
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\BANKNIFTY-"+file+".csv"):
            ddf = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\Quarterly_data\BANKNIFTY-"+file+".csv")
            ddf['Option_Type'] = ddf['ticker'].str[-2:]
            ddf['Strike'] = np.where((ddf['ticker'].str.len()==20) | (ddf['ticker'].str.len()==22) , ddf['ticker'].str[-6:-2] , ddf['ticker'].str[-7:-2])
            ddf['Symbol'] = 'BANKNIFTY' + 'QUARTERLY-' + file + + ddf['Strike'].astype(int).astype(str) + ddf['Option_Type']
            ddf['ticker'] = ddf['Symbol']
            ddf = ddf.drop(ddf.columns[9:],axis=1)
            ddf = ddf.rename(columns = {'date':'Date','ticker':'Ticker'})
            ddf = ddf.drop_duplicates()
            ddf.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+file+".csv",index=False)
    print("BANKNIFTY QUARTERLY CONTRACTS CREATED")
    
def banknifty_halfyearly():
    
    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv")

    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv")
        
    final_df = pd.DataFrame()
    file = 'I'
    for i in range(2):
        df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
        temp = df.copy()
        temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
        temp['Time'] = pd.to_datetime(temp['Time']).dt.time
        temp['Date'] = pd.to_datetime(temp['Date'])
        temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
        temp['Option_type'] = temp['Ticker'].str[-2:]
        temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
        temp["Temp"] = temp["Temp"].str[:-2]
        temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                    temp['Temp'].str[-5:],
                                    temp['Temp'].str[-4:])
        temp['Current_Year'] = temp['Date'].dt.year
        temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
        temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
        temp["Exp_month"] = temp["Temp"].str[2:5]
        temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
        temp = temp[(temp['Exp_month']=='JUN') | (temp['Exp_month']=='DEC')]
        temp = temp.reset_index(drop=True)
        final_df = final_df.append(temp)
        final_df = final_df.reset_index(drop=True)
        final_df = final_df.drop(final_df.columns[9:],axis=1)
        file+=file

    if final_df.empty==False:    
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-I' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY HALF-YEARLY-I CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-I.csv",index=False)


    final_df = pd.DataFrame()
    file = 'III'
    for i in range(2):
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv"):
            df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
            temp = df.copy()
            temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
            temp['Time'] = pd.to_datetime(temp['Time']).dt.time
            temp['Date'] = pd.to_datetime(temp['Date'])
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            temp['Option_type'] = temp['Ticker'].str[-2:]
            temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
            temp["Temp"] = temp["Temp"].str[:-2]
            temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                        temp['Temp'].str[-5:],
                                        temp['Temp'].str[-4:])
            temp['Current_Year'] = temp['Date'].dt.year
            temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
            temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
            temp["Exp_month"] = temp["Temp"].str[2:5]
            temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
            temp = temp[(temp['Exp_month']=='JUN') | (temp['Exp_month']=='DEC')]
            temp = temp.reset_index(drop=True)
            final_df = final_df.append(temp)
            final_df = final_df.reset_index(drop=True)
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            file='IV'
        else:
            print("BANKNIFTY-QUARTERLY-"+str(file)+' not found')
            file='IV'

    if final_df.empty==False:
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-II' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY HALF-YEARLY-II CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Half_Yearly\\Banknifty-II.csv",index=False)
    print("BANKNIFTY HALFYEARLY CONTRACTS CREATED")

def banknifty_yearly():
    if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv"):
        os.remove(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv")

    final_df = pd.DataFrame()
    file = 'I'
    add_file = 'I'
    for i in range(4):
        print(i,file)
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv"):  
            df = pd.read_csv(r"C:\Users\admin\Desktop\Pyspark_Contracts\BankNifty\\Quarterly_Data\BANKNIFTY-"+str(file)+".csv")
            temp = df.copy()
            temp = temp.rename(columns = {'ticker':'Ticker','date':'Date','time':'Time','open':'Open','low':'Low','high':'High','close':'Close','volume':'Volume'})
            temp['Time'] = pd.to_datetime(temp['Time']).dt.time
            temp['Date'] = pd.to_datetime(temp['Date'])
            temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
            temp['Option_type'] = temp['Ticker'].str[-2:]
            temp["Temp"] = temp["Ticker"].str.replace('BANKNIFTY',"")
            temp["Temp"] = temp["Temp"].str[:-2]
            temp["Strike"] = np.where((temp['Temp'].str.len()==12) | (temp['Temp'].str.len()==10),
                                        temp['Temp'].str[-5:],
                                        temp['Temp'].str[-4:])
            temp['Current_Year'] = temp['Date'].dt.year
            temp['Current_Year'] = temp['Current_Year'].astype(str).str[-2:]
            temp["Exp_year"] = np.where(temp['Temp'].str.len()==12,temp["Temp"].str[5:7],temp['Temp'].str[:2])
            temp["Exp_month"] = temp["Temp"].str[2:5]
            temp['Length_of_Temp'] = np.where(temp['Temp'].str.len()==12,12,temp['Temp'].str.len())
            temp = temp[(temp['Exp_month']=='DEC')]
            temp = temp.reset_index(drop=True)
            final_df = final_df.append(temp)
            final_df = final_df.reset_index(drop=True)
            final_df = final_df.drop(final_df.columns[9:],axis=1)
            if i==2:
                file='IV'
            else:
                file+=add_file

        else:
            print("BANKNIFTY QUARTERLY-"+str(file)+" does not exist")

    if final_df.empty==False:
        test = final_df.copy()
        test['Option_Type'] = test['Ticker'].str[-2:]
        test['Last'] = test['Ticker'].str[-7:]
        test['Strike'] = test['Last'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
        test['Symbol'] = test['Ticker'].str[:9] + '-I' + test['Strike'].astype(str) + test['Option_Type'].astype(str)
        test['Ticker'] = test['Symbol']
        test = test.drop(test.columns[9:13],axis=1)
        print("BANKNIFTY YEARLY-I CREATED")
        test.to_csv(r"C:\Users\admin\Desktop\Pyspark\BankNifty\\Yearly\\Banknifty-I.csv",index=False)

    else:
        print("Dataframe is empty!!!")
    print("BANKNIFTY YEARLY CONTRACTS CREATED")
    
print("BANKNIFTY CONTRACTS BEING CREATED")
banknifty_monthly()
banknifty_weekly()
banknifty_quarterly()
banknifty_halfyearly()
banknifty_yearly()

et = time.time()
print("\nBANKNIFTY CONTRACTS CREATED")
print("ELAPSED TIME",et-st)
        

r27022023
BANKNIFTY CONTRACTS BEING CREATED
0 I
1 II
2 III
3 IV
BANKNIFTY YEARLY-I CREATED
BANKNIFTY YEARLY CONTRACTS CREATED

BANKNIFTY CONTRACTS CREATED
ELAPSED TIME 0.3246302604675293


C:\Users\admin\AppData\Local\Temp\ipykernel_12476\1608901573.py:673: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(temp)
C:\Users\admin\AppData\Local\Temp\ipykernel_12476\1608901573.py:673: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(temp)
C:\Users\admin\AppData\Local\Temp\ipykernel_12476\1608901573.py:673: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(temp)
C:\Users\admin\AppData\Local\Temp\ipykernel_12476\1608901573.py:673: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(temp)
